# simple nn model fitting

Notebook will run a very simple nn on the training points

In [1]:
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
# import librosa
# import librosa.display
# import sklearn as skl
# import sklearn.utils
# import IPython.display as ipd
# from sklearn.model_selection import train_test_split
# import ast

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
#define network class
#used some help from https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

class Net(nn.Module):
    def __init__(self, input_size, output_size = 1):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(input_size, int(input_size/2))
        self.layer2 = nn.Linear(int(input_size/2), int(input_size/4))
        self.layer3 = nn.Linear(int(input_size/4), output_size)

    # x represents our data
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x

        

In [3]:
#import data
train_data = torch.tensor(np.load('data/train_data.npy').T).float()
labels = torch.tensor(np.load('data/train_labels.npy')).float()
n_dim = train_data.shape
print("n_dim = ", n_dim)
print('n_training_points = ', n_training_points )

n_dim =  50
n_training_points =  100000


In [11]:
model = Net(n_dim)
print(model)
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.MSELoss()

Net(
  (layer1): Linear(in_features=50, out_features=25, bias=True)
  (layer2): Linear(in_features=25, out_features=12, bias=True)
  (layer3): Linear(in_features=12, out_features=1, bias=True)
)


In [12]:
max_epochs = 2
for epoch in range(max_epochs):
    optimizer.zero_grad()   # zero the gradient buffers
    output = model(train_data.T.float())
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()    # Does the update
    
    print("loss = ", loss)

C:\Users\16kat\anaconda3\lib\site-packages\torch\nn\modules\loss.py:446: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([100000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\16kat\anaconda3\lib\site-packages\torch\autograd\__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


RuntimeError: Found dtype Double but expected Float